# buy-and-hold (monthly and holding period returns)

buy, then never ever sell, until the end date :)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *

import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

In [3]:
pf.DEBUG = True

Some global data

In [4]:
symbol = '^GSPC'
#symbol = 'SPY'
capital = 10000
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Define Strategy Class

In [5]:
class Strategy:

    def __init__(self, symbol, capital, start, end):
        self._symbol = symbol
        self._capital = capital
        self._start = start
        self._end = end

    def _algo(self):
        pf.TradeLog.cash = self._capital

        for i, row in enumerate(self._ts.itertuples()):

            date = row.Index.to_pydatetime()
            high = row.high; low = row.low; close = row.close 
            end_flag = pf.is_last_row(self._ts, i)
            shares = 0

            # buy
            if self._tlog.shares == 0:
                shares = self._tlog.buy(date, close)
            # sell
            elif end_flag:
                shares = self._tlog.sell(date, close)

            if shares > 0:
                pf.DBG("{0} BUY  {1} {2} @ {3:.2f}".format(
                       date, shares, self._symbol, close))
            elif shares < 0:
                pf.DBG("{0} SELL {1} {2} @ {3:.2f}".format(
                       date, -shares, self._symbol, close))

            # record daily balance
            self._dbal.append(date, high, low, close)

    def run(self):
        self._ts = pf.fetch_timeseries(self._symbol)
        self._ts = pf.select_tradeperiod(self._ts, self._start, self._end,
                                         use_adj=True)
        self._ts, self._start = pf.finalize_timeseries(self._ts, self._start)
        self._tlog = pf.TradeLog(self._symbol)
        self._dbal = pf.DailyBal()

        self._algo()

    def get_logs(self):
        """ return DataFrames """
        self.tlog = self._tlog.get_log()
        self.dbal = self._dbal.get_log(self.tlog)
        return self.tlog, self.dbal

    def get_stats(self):
        stats = pf.stats(self._ts, self.tlog, self.dbal, self._capital)
        return stats

Run Strategy

In [6]:
s = Strategy(symbol, capital, start, end)
s.run()

1927-12-30 00:00:00 BUY  566 ^GSPC @ 17.66
2020-07-17 00:00:00 SELL 566 ^GSPC @ 3224.73


Retrieve log DataFrames

In [7]:
tlog, dbal = s.get_logs()
stats = s.get_stats()

In [8]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
0,1927-12-30,17.66,2020-07-17,3224.73,3207.07,1815201.61,566,1815201.61,LONG,^GSPC


In [9]:
dbal.tail()

,high,low,close,shares,cash,leverage,state
date,,,,,,,
2020-07-13,1831195.60,1782581.78,1785858.94,566,4.44,1.00,-
2020-07-14,1811742.11,1770259.95,1809800.77,566,4.44,1.00,-
2020-07-15,1832870.94,1811634.61,1826237.43,566,4.44,1.00,-
2020-07-16,1822745.12,1810406.43,1820017.10,566,4.44,1.00,-
2020-07-17,1825201.61,1825201.61,1825201.61,0,1825201.61,1.00,X


In [10]:
pf.print_full(stats)

start                                                   1927-12-30
end                                                     2020-07-17
beginning_balance                                            10000
ending_balance                                          1825201.61
total_net_profit                                        1815201.61
gross_profit                                            1815201.61
gross_loss                                                    0.00
profit_factor                                                 1000
return_on_initial_capital                                 18152.02
annual_return_rate                                            5.79
trading_period                           92 years 6 months 17 days
pct_time_in_market                                          100.00
margin                                                           1
avg_leverage                                                  1.00
max_leverage                                                  

Summary

In [11]:
pf.summary(stats)

,strategy
annual_return_rate,5.79
max_closed_out_drawdown,-86.17
best_month,61.42
worst_month,-42.18
sharpe_ratio,0.39
sortino_ratio,0.49
monthly_std,5.35


In [12]:
returns = dbal['close']
pf.monthly_returns_map(returns['1990':])

Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
1990,-8.5,0.9,2.4,-2.7,9.2,-0.9,-0.5,-9.4,-5.1,-0.7,6.0,2.5,-8.2
1991,4.2,6.7,2.2,0.0,3.9,-4.8,4.5,2.0,-1.9,1.2,-4.4,11.2,26.3
1992,-2.0,1.0,-2.2,2.8,0.1,-1.7,3.9,-2.4,0.9,0.2,3.0,1.0,4.5
1993,0.7,1.0,1.9,-2.5,2.3,0.1,-0.5,3.4,-1.0,1.9,-1.3,1.0,7.1
1994,3.3,-3.0,-4.6,1.2,1.2,-2.7,3.1,3.8,-2.7,2.1,-4.0,1.2,-1.5
1995,2.4,3.6,2.7,2.8,3.6,2.1,3.2,-0.0,4.0,-0.5,4.1,1.7,34.1
1996,3.3,0.7,0.8,1.3,2.3,0.2,-4.6,1.9,5.4,2.6,7.3,-2.2,20.3
1997,6.1,0.6,-4.3,5.8,5.9,4.3,7.8,-5.7,5.3,-3.4,4.5,1.6,31.0
1998,1.0,7.0,5.0,0.9,-1.9,3.9,-1.2,-14.6,6.2,8.0,5.9,5.6,26.7
1999,4.1,-3.2,3.9,3.8,-2.5,5.4,-3.2,-0.6,-2.9,6.3,1.9,5.8,19.5


In [13]:
returns = dbal['close']
pf.holding_period_map(returns['1990':])